In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [29]:
from keras.models import Sequential
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, accuracy_score
from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
import plotly.graph_objs as go
from matplotlib.pyplot import cm
from keras.models import Model
import numpy as np
import keras
import h5py

In [20]:
with h5py.File("../input/3d-mnist/full_dataset_vectors.h5", "r") as hf:    
     X_train = hf["X_train"][:]
     y_train = hf["y_train"][:]    
     X_test = hf["X_test"][:]  
     y_test = hf["y_test"][:] 

In [21]:
print("X_train shape :", (X_train.shape))
print("X_test shape :" ,(X_test.shape))
print("Y_train shape :" ,(y_train.shape))
print("Y_test shape :" ,(y_test.shape))


In [22]:
xtrain=np.ndarray((X_train.shape[0],4096,3))
xtest=np.ndarray((X_test.shape[0],4096,3))

# not exactly clear what this block is doing
def add_rgb_dimension(array):
    scaler_map = cm.ScalarMappable(cmap="Oranges")
    array = scaler_map.to_rgba(array)[:, : -1]
    return array
for i in range(X_train.shape[0]):
    xtrain[i] = add_rgb_dimension(X_train[i])
for i in range(X_test.shape[0]):
    xtest[i] = add_rgb_dimension(X_test[i])

## convert to 1 + 4D space (1st argument represents number of rows in the dataset)
xtrain = xtrain.reshape(X_train.shape[0], 16, 16, 16, 3)
xtest = xtest.reshape(X_test.shape[0], 16, 16, 16, 3)

## convert target variable into one-hot
y_train = keras.utils.np_utils.to_categorical(y_train, 10)
y_test = keras.utils.np_utils.to_categorical(y_test, 10)



In [23]:
print(xtrain.shape)

In [24]:
# One Input layer with dimentions 16, 16, 16, 3
# Output layer with dimentions 10
# Apply 4 Convolutional layer with increasing order of filter size (standard size : 8, 16, 32, 64) and fixed kernel size = (3, 3, 3)
# Apply 2 Max Pooling layers, one after 2nd convolutional layer and one after fourth convolutional layer.
# Batch normalization on convolutiona architecture
# Dense layers with 2 layers followed by dropout to avoid overfitting

In [32]:
sample_shape=(16,16,16,3)
model=Sequential()
model.add(Conv3D(filters=8,kernel_size=(3,3,3),activation='relu',input_shape=sample_shape))
model.add(Conv3D(filters=16,kernel_size=(3,3,3),activation='relu'))
model.add(MaxPool3D(pool_size=(2,2,2)))
model.add(Conv3D(filters=32,kernel_size=(3,3,3),activation='relu'))
model.add(Conv3D(filters=64,kernel_size=(3,3,3),activation='relu'))
model.add(MaxPool3D(pool_size=(2,2,2)))
model.add(BatchNormalization())
#dense layer requires input in single-dimensional shape i.e. 1-D array.
model.add(Flatten())
model.add(Dense(2048,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(10,activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])
model.summary()
model.fit(x=xtrain, y=y_train, batch_size=128, epochs=50, validation_split=0.2)


          

In [34]:
model.predict(xtest)